In [127]:
import networkx as nx
import matplotlib.pyplot as plt

In [128]:
class Node:

    node_list = []

    def __init__(self, x_pos: int, y_pos: int, logo: str, driving=False) -> None:
        self.x = x_pos
        self.y = y_pos
        self.logo = logo
        # type/logo could be an enum
        self.node_list.append(self)
        self.driving = driving
    
    def placeOnGrid(self, dataframe):
        dataframe[self.x][self.y] = self.logo
        return dataframe
    
    # @abstractmethod
    def checkAccess(self):
        pass

class Building(Node):

    def __init__(self, x_pos: int, y_pos: int, logo: str, driving: bool) -> None:
        super().__init__(x_pos, y_pos, logo, driving=False)

    def checkAccess(self):
        pass
    

class Parking(Node):

    parking_list = []
    
    def __init__(self, x_pos: int, y_pos: int, logo: str) -> None:
            super().__init__(x_pos, y_pos, logo, driving=True)

    def checkAccess(self):
            pass

class Footpath(Node):

    footpath_list = []

    def __init__(self, x_pos: int, y_pos: int, logo: str) -> None:
            super().__init__(x_pos, y_pos, logo, driving=False)

    def checkAccess(self):
        pass


In [129]:
shop = Building(1,2,logo='S', driving=True)
footpath1 = Footpath(1,5,logo='F')
footpath2 = Footpath(6,5,logo='F')
parking= Parking(7,9,logo='P')


In [130]:
grid_walking = nx.Graph()

# Add nodes
grid_walking.add_node(shop)
grid_walking.add_node(footpath1)
grid_walking.add_node(footpath2)
grid_walking.add_node(parking)


In [131]:
for node in grid_walking.nodes:
    print(node.logo, node.x, node.y, node)


S 1 2 <__main__.Building object at 0x7ff07a64a0d0>
F 1 5 <__main__.Footpath object at 0x7ff07a64a4c0>
F 6 5 <__main__.Footpath object at 0x7ff07a64a2b0>
P 7 9 <__main__.Parking object at 0x7ff07a64a880>


In [132]:
grid_walking.number_of_nodes()

4

### Add edges

In [133]:
from scipy.spatial import distance

def euclidean_distance(node1, node2):
   x1, y1 = node1.x, node1.y
   x2, y2 = node2.x, node2.y
   return distance.euclidean((x1, y1), (x2, y2))
   #
   # return nx.shortest_path_length(grid_walking, node1, node2, weight="distance")

euclidean_distance(shop, parking)

9.219544457292887

In [134]:
class Edge:
    
        def __init__(self, node1, node2) -> None:
            self.node1 = node1
            self.node2 = node2
            self.distance = euclidean_distance(node1, node2)

In [135]:
grid_walking.add_edge(shop, footpath1, distance=euclidean_distance(shop, footpath1))
grid_walking.add_edge(footpath1, footpath2, distance=euclidean_distance(footpath1, footpath2))
grid_walking.add_edge(footpath2, parking, distance=euclidean_distance(footpath2, parking))

In [147]:
def walking_distance(node1, node2):
    parameters = (grid_walking, node1, node2)
    return round(nx.shortest_path_length(*parameters, weight="distance"), 2)

In [148]:
walking_distance(shop, parking)

12.12

### Add driving distance

In [138]:
grid_driving = nx.Graph()

for node in Node.node_list:
    grid_driving.add_node(node)

for node in grid_driving.nodes:
    print(node.logo, node.x, node.y)

grid_driving.add_edge(shop, parking, distance=4)



S 1 2
F 1 5
F 6 5
P 7 9


In [140]:
def driving_distance(node1, node2):
    parameters = (grid_driving, node1, node2)
    return round(nx.shortest_path_length(*parameters, weight="distance"), 2)


driving_distance(shop, parking)

4